In [1]:
using Distributed
addprocs(44);

### Fitting a choice model.

First you have to include the necessary package

In [2]:
using pulse_input_DDM

┌ Info: Recompiling stale cache file /usr/people/briandd/.julia/compiled/v1.2/pulse_input_DDM/m8WoA.ji for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
└ @ Base loading.jl:1240
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `pulse_input_DDM.load :: Tuple{String}` in module `pulse_input_DDM`
└ @ Base.Docs docs/Docs.jl:223


### Geneerate some data
Blah blah blah

In [3]:
f, ncells, ntrials, nparams = "Softplus", [5,4], [300,400], 3

θ_syn = θneural(θz = θz(σ2_i = 0.5, B = 40., λ = -0.5, σ2_a = 50., σ2_s = 0.1,
    ϕ = 0.6, τ_ϕ =  0.02),
    θy=[[Softplus(10., rand([-1.,1.]),0.) for n in 1:N] for N in ncells], ncells=ncells,
    nparams=nparams, f=f)

θneural{θz{Float64},Array{Array{Softplus{Float64},1},1}}
  θz: θz{Float64}
  θy: Array{Array{Softplus{Float64},1}}((2,))
  ncells: Array{Int64}((2,)) [5, 4]
  nparams: Int64 3
  f: String "Softplus"


In [4]:
data = synthetic_data(θ_syn, ntrials; centered=false);

In [5]:
data[1][1]

neuraldata
  input_data: pulse_input_DDM.neuralinputs{pulse_input_DDM.clicks,pulse_input_DDM.binned_clicks}
  spikes: Array{Array{Int64,1}}((5,))
  ncells: Int64 5


In [6]:
using MAT

In [7]:
centered, dt = true, 1e-2

(true, 0.01)

In [8]:
data = read(matopen("/usr/people/briandd/Projects/neural_DDM_analysis/data/hanks_data_sessions/B068_52307.mat"), "rawdata")

T = vec(data["T"])
L = vec(map(x-> vec(collect(x)), data[collect(keys(data))[occursin.("left", collect(keys(data)))][1]]))
R = vec(map(x-> vec(collect(x)), data[collect(keys(data))[occursin.("right", collect(keys(data)))][1]]))

click_times = pulse_input_DDM.clicks.(L, R, T)
binned_clicks = pulse_input_DDM.bin_clicks(click_times, centered=centered, dt=dt)

ncells = size(data["spike_times"][1], 2)

1

In [9]:
spike_times = vec(map(x-> vec(vec.(collect.(x))), data["spike_times"]));

In [10]:
output = map((spike_times, binned_clicks)-> pulse_input_DDM.bin_spikes(spike_times, dt, binned_clicks.nT), spike_times, binned_clicks)
trial = getindex.(output, 1)
padded = getindex.(output, 2)

600-element Array{Array{Array{Int64,1},1},1}:
 [[0, 1, 0, 0, 0, 1, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
 ⋮                                                                
 [[0, 0, 0, 0, 0

In [12]:
μ_rnt = pulse_input_DDM.filtered_rate.(padded, dt)

UndefVarError: UndefVarError: KernelFactors not defined

In [ ]:
μ_t = map(n-> [max(0., mean([μ_rnt[i][n][t]
    for i in findall(nT .>= t)]))
    for t in 1:(maximum(nT))], 1:ncells)

λ0 = map(binned_clicks-> bin_λ0(μ_t, dt, binned_clicks.nT), binned_clicks)

input_data = neuralinputs(click_times, binned_clicks, λ0, dt, centered)
neuraldata(input_data, spikes, ncells)

In [10]:
data = read(matopen("/usr/people/briandd/Projects/neural_DDM_analysis/data/hanks_data_sessions/B068_52307.mat"), "rawdata")

Dict{String,Any} with 9 entries:
  "rightbups"   => Any[[0.0 0.306165 … 0.385765 0.394515] [0.0 0.00826 … 0.1287…
  "cellID"      => Any[2261.0 2261.0 … 2261.0 2261.0]
  "T"           => Any[0.445579 0.186189 … 0.461016 0.358184]
  "stim_start"  => Any[0.554618 0.814012 … 0.538996 0.642017]
  "leftbups"    => Any[[0.0 0.02087 … 0.398585 0.414005] 0.0 … [0.0 0.046735 … …
  "spike_times" => Any[Any[[-1.74085; -1.69176; … ; 3.29756; 3.38327]] Any[[-0.…
  "pokedR"      => Any[0.0 1.0 … 0.0 1.0]
  "correct_dir" => Any[0.0 1.0 … 0.0 1.0]
  "cpoke_end"   => Any[1.0002 1.0002 … 1.00001 1.0002]

In [41]:
using StatsBase

In [43]:
spike_times[1]

1-element Array{Array{Float64,1},1}:
 [-1.7408498369477408, -1.691760167348543, -1.587343870123727, -1.47967059481743, -0.13460464779927372, -0.08631297282408434, -0.040724279660930396, 0.06212502811104548, 0.15115142891955236, 0.19406714007346215  …  2.7403380023752155, 2.7847287036020183, 2.8310843916042407, 2.878178074639441, 2.9723034411290477, 3.0235750960419003, 3.1361623382727024, 3.2105968372925466, 3.2975642519559236, 3.3832726750952182]

In [50]:
dt = 1e-2
nT = ceil(Int,click_times[1].T/dt)

45

In [54]:
map(x-> fit(Histogram, vec(collect(x .- 0.)), collect(range(0,stop=nT*dt,length=nT+1)), closed=:left).weights, spike_times[1])

1-element Array{Array{Int64,1},1}:
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [23]:
n = 303

303

In [24]:
loglikelihood(θ_syn, data, n)

-493204.7825872947

In [16]:
θy0 = vcat(vcat(initialize_θy.(data, f)...)...)

θy0

3-element Array{Float64,1}:
 9.868493150684932 
 4.1611487103704405
 0.0               

In [17]:
options0 = neuraloptions(ncells=ncells,
    fit=vcat(falses(dimz), trues(sum(ncells)*nparams)),
    x0=vcat([0., 30., 0. + eps(), 0., 0., 1. - eps(), 0.008], θy0),
    nparams=nparams, f=f, 
    lb = vcat([0., 8., -5., 0., 0., 0.01, 0.005], repeat([0.,-10.,-10.,], sum(ncells))), 
    ub = vcat([2., 30., 5., 100., 2.5, 1.2, 1.], repeat([100.,10.,10.], sum(ncells))))

neuraloptions
  ncells: Array{Int64}((1,)) [1]
  nparams: Int64 3
  f: String "Softplus"
  fit: Array{Bool}((10,)) Bool[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
  lb: Array{Float64}((10,)) [0.0, 8.0, -5.0, 0.0, 0.0, 0.01, 0.005, 0.0, -10.0, -10.0]
  ub: Array{Float64}((10,)) [2.0, 30.0, 5.0, 100.0, 2.5, 1.2, 1.0, 100.0, 10.0, 10.0]
  x0: Array{Float64}((10,)) [0.0, 30.0, 2.220446049250313e-16, 0.0, 0.0, 0.9999999999999998, 0.008, 9.868493150684932, 4.1611487103704405, 0.0]


### Optimize stuff
Blah blah blah

In [18]:
model = optimize(data, options0)

Fminbox
-------
Initial mu = 76.8591

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 76.8591

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     1.028419e+06     2.840945e+04
 * time: 4.601478576660156e-5
     1     9.979438e+05     2.504234e+04
 * time: 0.856982946395874
     2     9.745756e+05     2.263427e+04
 * time: 1.5163750648498535
     3     9.544536e+05     2.067304e+04
 * time: 2.0615010261535645
     4     9.344140e+05     1.881528e+04
 * time: 2.604465961456299
     5     9.211516e+05     1.763576e+04
 * time: 3.7332401275634766
     6     9.071323e+05     1.642870e+04
 * time: 4.333832025527954
     7     8.979307e+05     1.565860e+04
 * time: 4.908924102783203
     8     8.882255e+05     1.486328e+04
 * time: 5.509658098220825
     9     8.813918e+05     1.431444e+04
 * time: 6.083688020706177
    10     8.740911e+05     1.373654e+04
 * time: 6.657217979431152
    11     8.684795e+05     1.329932e+0

neuralDDM{θneural{θz{Float64},Array{Array{Softplus{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural{θz{Float64},Array{Array{Softplus{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((1,))


In [58]:
fit=vcat(falses(dimz-2), falses(2), trues(sum(ncells)*nparams))
#x0=vcat([0.5, 8., -2., 10., 1., 0.4, 0.008], pulse_input_DDM.flatten(model.θ)[dimz+1:end])
x0=vcat([θ_syn.θz.σ2_i, θ_syn.θz.B, θ_syn.θz.λ, θ_syn.θz.σ2_a, θ_syn.θz.σ2_s, θ_syn.θz.ϕ, θ_syn.θz.τ_ϕ], 
    pulse_input_DDM.flatten(model.θ)[dimz+1:end]);

In [8]:
pulse_input_DDM.flatten(θ_syn)

10-element Array{Float64,1}:
  0.01              
 40.0               
 -1.0               
 50.0               
  0.01              
  0.9999999999999998
  0.02              
 10.0               
 10.0               
 10.0               

In [26]:
fit=vcat(falses(2),trues(1),falses(4), falses(1), trues(1), falses(1))
#x0=vcat([0.5, 8., -2., 10., 1., 0.4, 0.008], pulse_input_DDM.flatten(model.θ)[dimz+1:end])
x0=pulse_input_DDM.flatten(θ_syn);

In [27]:
options = neuraloptions(ncells=ncells, x0=x0, fit=fit, nparams=nparams, f=f, 
    lb = vcat([0., 8., -5., 0.,   0.,  0.01, 0.005], repeat([0.,-100.,-100.,], sum(ncells))), 
    ub = vcat([2., 50., 5., 100., 2.5, 1.2,  1.], repeat([100.,100.,100.], sum(ncells))))
model = optimize(data, options, n)

Fminbox
-------
Initial mu = 0.405546

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.405546

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     4.931998e+05     1.424261e+02
 * time: 8.893013000488281e-5
     1     4.931920e+05     1.587904e+02
 * time: 89.6972451210022
     2     4.931687e+05     3.646034e+00
 * time: 179.10835194587708
     3     4.931686e+05     3.102588e-01
 * time: 238.4584789276123
     4     4.931686e+05     1.356555e-02
 * time: 297.5549569129944

Exiting inner optimizer with x = [-0.8456257015853647, 9.536915693415581]
Current distance to box: 4.15437
Decreasing barrier term μ.

Fminbox iteration 2
-------------------
Calling inner optimizer with mu = 0.000405546

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     4.931737e+05     4.178036e-02
 * time: 5.2928924560546875e-5
     1     4.931737e+05     9.031785e-03
 * time: 177.1871218681335

neuralDDM{θneural{θz{Float64},Array{Array{Softplus{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural{θz{Float64},Array{Array{Softplus{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((1,))


In [21]:
pulse_input_DDM.flatten(model.θ)

10-element Array{Float64,1}:
  0.01              
 40.0               
 -0.9795359665530979
 50.0               
  0.01              
  0.9999999999999998
  0.02              
 10.0               
  9.923376873524658 
 10.0               

In [28]:
pulse_input_DDM.flatten(model.θ)

10-element Array{Float64,1}:
  0.01              
 40.0               
 -0.8456482038525149
 50.0               
  0.01              
  0.9999999999999998
  0.02              
 10.0               
  9.536945285154058 
 10.0               

### Compute Hessian
Blah blah blah

In [61]:
# ### Compute Hessian and the confidence interavls
# Blah blah blah

H = Hessian(model, n)
CI, HPSD = CIs(H);

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 0.5554666390572115
└ @ pulse_input_DDM /usr/people/briandd/Projects/pulse_input_DDM/src/base_model.jl:19


In [62]:
CI

34-element Array{Float64,1}:
  7.8969487286689946  
 30.64572976694986    
  0.5227482237011131  
 57.30293808493733    
  1.052098579676006   
  0.2854240586074219  
  0.012721591348291143
  1.0463482501293424  
  0.44395425868206206 
  2.0257895431530937  
  0.9426157927375336  
  0.475480730183873   
  2.4824442795023702  
  ⋮                   
  0.8049862106153354  
  0.46914692848536876 
  1.8033429981739468  
  0.7664899738381734  
  0.6005794705927137  
  3.6429205869982546  
  0.7452265816163378  
  0.5191335535916887  
  2.378824818494355   
  0.8853759123766897  
  0.4566170373515372  
  2.631376416360695   

In [63]:
hcat(collect(pulse_input_DDM.flatten(model.θ)) - CI, collect(pulse_input_DDM.flatten(θ_syn)), collect(pulse_input_DDM.flatten(model.θ)) + CI)

34×3 Array{Float64,2}:
 -7.39695      0.5     8.39695  
  9.35427     40.0    70.6457   
 -1.02275     -0.5     0.0227482
 -7.30294     50.0   107.303    
 -0.952099     0.1     1.1521   
  0.314576     0.6     0.885424 
  0.00727841   0.02    0.0327216
  7.92373     10.0    10.0164   
 -1.23579     -1.0    -0.347883 
  0.123542     0.0     4.17512  
  9.0495      10.0    10.9347   
 -1.35359     -1.0    -0.402627 
 -2.15134      0.0     2.81355  
  ⋮                             
  8.62564     10.0    10.2356   
  0.394949     1.0     1.33324  
 -0.606459     0.0     3.00023  
  9.55002     10.0    11.083    
  0.409178     1.0     1.61034  
 -5.67385      0.0     1.61199  
  9.42011     10.0    10.9106   
 -1.49154     -1.0    -0.453278 
 -3.13858      0.0     1.61907  
  9.44594     10.0    11.2167   
  0.365056     1.0     1.27829  
 -2.59983      0.0     2.66292  

In [64]:
hcat(collect(pulse_input_DDM.flatten(model.θ)), collect(pulse_input_DDM.flatten(θ_syn)))

34×2 Array{Float64,2}:
  0.5         0.5 
 40.0        40.0 
 -0.5        -0.5 
 50.0        50.0 
  0.1         0.1 
  0.6         0.6 
  0.02        0.02
  8.97008    10.0 
 -0.791837   -1.0 
  2.14933     0.0 
  9.99211    10.0 
 -0.878107   -1.0 
  0.331102    0.0 
  ⋮               
  9.43063    10.0 
  0.864096    1.0 
  1.19688     0.0 
 10.3165     10.0 
  1.00976     1.0 
 -2.03093     0.0 
 10.1653     10.0 
 -0.972411   -1.0 
 -0.759752    0.0 
 10.3313     10.0 
  0.821673    1.0 
  0.0315479   0.0 

In [65]:
hcat(collect(pulse_input_DDM.flatten(model.θ)) - CI .< 
    collect(pulse_input_DDM.flatten(θ_syn)) .< 
    collect(pulse_input_DDM.flatten(model.θ)) + CI)

34×1 BitArray{2}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1